In [1]:
import scipy as sp
import numpy as np
from numbers import Number

In [ ]:
def shipping_analysis(A, b, M, kra=True, eq=True, flag=True):
    """
    Compute minimising solution for relevant optimisation problems related to
    the congestion network as defined in the coursework.
    ===
    Inputs:
    A (numpy.ndarray) : Congestion gradient coefficients of cost functions in a
                        2D matrix. Must be a diagonal matrix.
    b (numpy.ndarray) : Constant coefficients of cost functions in a 1D vector.
    M (numbers.Number): Total mass in the network.
    kra (bool)        : Determine whether to include the Kra Canal or not.
    eq (bool)         : Determine optimisation problem to solve: equilibrium
                        solution if True, social optimum if False.
    flag (bool)       : if True, will print information about solution.
    ---
    Output:
    constrained (numpy.ndarray): Flow that minimises the optimisation problem.
    ===
    """
    if not (isinstance(A, np.ndarray) and len(A.shape) == 2
            and isinstance(b, np.ndarray) and len(b.shape) == 1
            and isinstance(M, Number)):
        raise TypeError("A has to be a 2D numpy array, b a 1D numpy array"
                        "and M a number.")
    
    if kra:
        A_w = A
        b_w = b
        C = np.array([[1, 1, 0, 0, 0, 0],
                      [-1, 0, 1, 0, 0, 1],
                      [0, -1, 0, 1, 1, 0]])
    else:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
        C = np.array([[1, 1, 0, 0, 0],
                      [-1, 0, 1, 0, 0],
                      [0, -1, 0, 1, 1]])
    d = np.array([M, 0, 0])
    if not eq:
        A_w *= 2

    # Unconstrained solution
    A_1 = np.diag(1/np.diag(A_w))
    un = -A_1 @ b_w
    
    # Constrained optimization with non-negativity and equality constraints
    def objective(x):
        return 0.5 * x @ A_w @ x + b_w @ x
    
    def equality_constraints(x):
        return C @ x - d

    constraints = [{'type': 'eq', 'fun': equality_constraints}]
    bounds = [(0, None) for _ in range(len(b_w))]  # Non-negativity constraints
    result = sp.optimize.minimize(objective, un,
                                  bounds=bounds, constraints=constraints)
    if not result.success:
        raise ValueError("Optimization failed to converge.")
    constrained = result.x
    if flag:
        if eq:
            text = "Equilibrium Analysis"
        else:
            text = "Social Optimum Analysis"
        if kra:
            text += " (With Kra Canal)"
        else:
            text += " (Without Kra Canal)"
        print(text)
        print(f"Optimal Solution: {constrained}")
    
    return constrained

# Example usage
A = np.diag([1, 1, 5, 3, 1.5, 20])
b = np.array([10, 20, 15, 14, 32, 10])
M = 100
x = shipping_analysis(A, b, M, kra=False, eq=True, flag=True)

Equilibrium Analysis (Without Kra Canal)
Optimal Solution: [27.62499745 72.37500255 27.62499745 28.12497573 44.25002682]


In [82]:
congestion = 1 / np.array([17.24, 25.4, 36.19, 0.04])
congestion = congestion / np.sum(congestion)
congestion = np.concatenate(([0.0000000001, 0.000000001], congestion))
A = np.diag(congestion)
b = np.array([118093.97, 230596.48, 159620.44, 143039.42, 315912.85, 103749.65])
b /= np.sum(b)
x = shipping_analysis(A, b, M, kra=True, eq=True, flag=True)

Equilibrium Analysis (With Kra Canal)
Optimal Solution: [6.36593078e+01 3.63406922e+01 6.34596424e+01 3.63406922e+01
 5.32907052e-15 1.99665347e-01]


In [76]:
def route_cost(x):
    """
    Compute the total cost of traversing each of the routes of the
    congestion network from Node A to Node D, given flow x.
    ===
    Inputs:
    x (numpy.ndarray): Flow of system. Must be a 5 or 6 dimensional vector.
    ---
    Outputs:
    c (numpy.ndarray): Cost of each route.
    ===
    """
    if len(x) == 6:
        A_w = A
        b_w = b
        r = np.array([[1, 0, 0, 0, 0, 1],
                      [1, 0, 1, 0, 0, 0],
                      [0, 1, 0, 1, 0, 0],
                      [0, 1, 0, 0, 1, 0]])
    elif len(x) == 5:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
        r = np.array([[1, 0, 1, 0, 0],
                      [0, 1, 0, 1, 0],
                      [0, 1, 0, 0, 1]])
    else:
        raise ValueError('x needs to be 5 or 6 dimensional.')
    return r @ (A_w @ x + b_w)

def average_cost(x):
    """
    Compute the average cost of any given user of the congestion network
    from Node A to Node D, given flow x.
    ===
    Inputs:
    x (numpy.ndarray): Flow of system. Must be a 5 or 6 dimensional vector.
    ---
    Outputs:
    c (numpy.ndarray): Average cost per single user of network.
    ===
    """
    if len(x) == 6:
        A_w = A
        b_w = b
    elif len(x) == 5:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
    return (x.T @ A_w @ x + b_w .T @ x) / M
cost = average_cost(x)

In [75]:
route_cost(x)

array([190.75, 190.75, 190.75])

In [ ]:
def POA(x_bar, x_tilde):
    """
    Calculate Price of Anarchy of the congestion network.
    ===
    Inputs:
    x_bar (numpy.ndarray):   Flow of system in equilibrium state.
    x_tilde (numpy.ndarray): Flow of system in social optimum.
    ---
    Outputs:
    c (numpy.ndarray): Average cost per single user of network.
    ===
    """
    if x_bar.shape != x_tilde.shape:
        raise ValueError("x_bar and x_tilde must be vectors of the same size.")
    return average_cost(x_bar)/average_cost(x_tilde)

POA(shipping_analysis(A, b, M, kra=False, eq=True, flag=False),
    shipping_analysis(A, b, M, kra=False, eq=False, flag=False))

np.float64(1.0000186471036423)